In [2]:
import torch 
import torchvision.datasets as datasets
from PIL.ImageShow import show
from PIL.Image import Image
import numpy as np

In [3]:
# Load MNIST data set
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=None)

# Get first image data and target and display image
train_image_zero, train_target_zero = train_data[0]
image_zero_pixels = train_image_zero.getdata()

train_image_zero.show()
print(f"The first label is a {train_target_zero}.")
print(f"There are {len(image_zero_pixels)} pixels in the image.")

The first label is a 5.
There are 784 pixels in the image.


In [17]:
alpha = 0.01              

def w_sum(inputs, weights):
    if len(inputs) != len(weights):
        raise ValueError("Vectors need to be of same len")
    output = 0
    
    for input, weight in zip(inputs, weights):
        output += input * weight
    return output

def neural_network(inputs, weights, target):
    pred = w_sum(inputs, weights)
    error = (target - pred) ** 2
    delta = pred - target
    return pred, error, delta
    
def ele_mul(scalar, vector):
    output = np.zeros(len(vector))

    for i in range(len(vector)):
        output[i] = vector[i] * scalar
    
    return output

def forward_propagation(inputs, weights, target):
    pred, error, delta = neural_network(inputs, weights, target)
    weight_deltas = ele_mul(delta, inputs)                          # Why scale input by delta?         
    weights -= alpha * weight_deltas            # Backward propagtion
    return pred, error



In [18]:
# Initialise weights and test run
# Note it is only a single neuron
weights = np.random.uniform(-0.5, 0.5, size=784)

for i in range(len(train_data)):
    input_data, target = train_data[i]
    input_data = np.array(input_data.getdata()) / 255.0

    pred, error = forward_propagation(input_data, weights, target)

    if i % 1000 == 0: 
            print(f"Iteration {i}: Prediction: {pred}, Error: {error}")


Iteration 0: Prediction: 1.256459932033792, Error: 14.014092240468441
Iteration 1000: Prediction: 2.320810168563338, Error: 5.38615983850699
Iteration 2000: Prediction: 5.513675525744381, Error: 0.2638625457487658
Iteration 3000: Prediction: 5.046140065675671, Error: 15.633008380255184
Iteration 4000: Prediction: 9.02277533450218, Error: 4.091620053870406
Iteration 5000: Prediction: 7.294402812251568, Error: 0.08667301586163204
Iteration 6000: Prediction: 6.157701275758545, Error: 0.024869692375872717
Iteration 7000: Prediction: 7.2259921741924575, Error: 0.599088114411319
Iteration 8000: Prediction: 4.289160577814207, Error: 18.3968984622755
Iteration 9000: Prediction: 10.056133246445585, Error: 16.452216912921198
Iteration 10000: Prediction: 4.562339498626215, Error: 2.440904708967612
Iteration 11000: Prediction: 4.817591335816604, Error: 1.3980902491359628
Iteration 12000: Prediction: 3.6239731066329623, Error: 11.397557584737491
Iteration 13000: Prediction: 5.235209335174055, Error